In [80]:
import pandas as pd             
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

sc = SparkContext.getOrCreate()

TRAIN_TEST = 0.8

In [79]:
random.random()

0.08003211248130271

In [82]:
def preprocessing(row):
    #splitting
    split=row.split(" ")
    #del 56 col
    del split[56]
    #assign random key to the datapoint
    key=random.getrandbits(64)
    #assign train/test 
    if(random.random()<TRAIN_TEST):
        train=1
    else:
        train=0    
    #return
    return (key,train,split)

In [99]:
dataset=sc.textFile("dataset/spam.data").map(preprocessing).sortBy(lambda x: x[0])
train=dataset.filter(lambda x: x[1]==1)
test=dataset.filter(lambda x: x[1]==0)
print("Split train/test done. Train contains "+str(train.count())+" elements, Test contains "+str(test.count())+" elements")

Split train/test done. Train contains 3652 elements, Test contains 949 elements


In [97]:
train.count()

3684

In [2]:
dataset=pd.read_csv("dataset/spam.data", sep=' ', dtype='float64', index_col=False, names=['f'+str(i) for i in range(0,58)])
dataset=dataset.drop("f56", axis=1)

dataset.head(5)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f47,f48,f49,f50,f51,f52,f53,f54,f55,f57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61.0,1.0
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101.0,1.0
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485.0,1.0
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40.0,1.0
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40.0,1.0


In [3]:
dataset_RDD=sc.parallelize(dataset)
X=sc.parallelize(dataset.drop("f57", axis=1))
y=sc.parallelize(dataset["f57"])


In [4]:


X=preprocessing.normalize(dataset.drop("f57", axis=1), norm='l2')
y=dataset["f57"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)